In [1]:
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import shap

In [ ]:
# fetch dataset 
credit_score = pd.read_csv("../../data/credit_score.csv")
credit_score['R_EXPENDITURE_SQR'] = credit_score['R_EXPENDITURE']**2

# data (as pandas dataframes) 
X = credit_score[['INCOME','R_DEBT_INCOME','R_EXPENDITURE','R_EXPENDITURE_SQR','R_EXPENDITURE_SAVINGS','R_ENTERTAINMENT','CAT_GAMBLING']].copy()

# one-hot encoding
X['GAMBLING_LOW'] = X['CAT_GAMBLING'].apply(lambda x: 1 if x == 'Low' else 0)
X['GAMBLING_HIGH']  = X['CAT_GAMBLING'].apply(lambda x: 1 if x == 'High' else 0)
X.drop(columns=['CAT_GAMBLING'], inplace=True)

y = credit_score['DEFAULT']

# get a balanced dataset
n = np.sum(y)
X = pd.concat([X[y==1],X[y==0].sample(n=n, random_state=0)])
y = pd.concat([y[y==1],y[y==0].sample(n=n, random_state=0)])

print(n,len(X),len(y))


284 568 568


In [3]:
# fit model
model = sm.Logit(y, X).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.636155
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                DEFAULT   No. Observations:                  568
Model:                          Logit   Df Residuals:                      560
Method:                           MLE   Df Model:                            7
Date:                Wed, 16 Oct 2024   Pseudo R-squ.:                 0.08222
Time:                        15:16:21   Log-Likelihood:                -361.34
converged:                       True   LL-Null:                       -393.71
Covariance Type:            nonrobust   LLR p-value:                 1.694e-11
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
INCOME                -5.557e-07    8.7e-07     -0.639      0.523   -2.26e-06    1.15e-06
R_DEBT_INCOME             0.1147      0.018      6.468      0.000       0.080       0.149
R_EXPENDITURE            -7.3747      3.140     -2.349      0.019     -13.529      -1.221
R_EXPENDITURE_SQR         9.1949      3.595      2.558      0.011       2.149      16.241
R_EXPENDITURE_SAVINGS     0.0644      0.053      1.219      0.223      -0.039       0.168
R_ENTERTAINMENT           0.8753      1.363      0.642      0.521      -1.796       3.547
GAMBLING_LOW              0.2376      0.299      0.794      0.427      -0.349       0.824
GAMBLING_HIGH             0.2490      0.219      1.138      0.255      -0.180       0.678
=========================================================================================
"""

In [4]:
# calculate model accuracy
model_pred = model.predict(X)
model_pred = model_pred.apply(lambda x: 1 if x > 0.5 else 0)
model_accuracy = np.mean(model_pred == y)
print("Model accuracy: ", model_accuracy)

# model confusion matrix
model_cm = pd.crosstab(model_pred, y, rownames=['Predicted'], colnames=['Actual'])
model_cm


Model accuracy:  0.6285211267605634


Actual,0,1
Predicted,,
0,196,123
1,88,161
